In [76]:
import pandas as pd
import numpy as np

1\. Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named "data_int.txt". Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named "data_float.txt". Use the `cat` command to print the content of the file.
+ load the txt file of the previous point and convert it to a csv file by hand.

In [81]:
integ = [i for i in range(50)]
with open('data_int.txt', 'w') as f:
    f.write(str(integ)) #data need to be converted to string
    
#!type data_int.txt # type is the equivalent of cat command for windows

matr = np.array([float(i) for i in range(1,26)]).reshape(5,5)
with open('data_float.txt','w') as g:
    g.write(str(matr))
    
#!type data_float.txt

import csv

with open('data_float.txt','r') as h:
    m = h.readlines()
#print(m)
m = [m[i].replace("\n","") for i in range(len(m))]
m = [m[i].replace("[","") for i in range(len(m))]
m = [m[i].replace("]","") for i in range(len(m))]
n = []*25

#print(m)
with open('data_float.csv','w', newline = "\n") as i:
    csv.writer(i, delimiter = ".").writerows(m)
    
!type data_float.csv 
#data is not represented correctly due to the delimiter

 .1.".". . .2.".". . .3.".". . .4.".". . .5."."
 . .6.".". . .7.".". . .8.".". . .9.".". .1.0."."
 .1.1.".". .1.2.".". .1.3.".". .1.4.".". .1.5."."
 .1.6.".". .1.7.".". .1.8.".". .1.9.".". .2.0."."
 .2.1.".". .2.2.".". .2.3.".". .2.4.".". .2.5."."


2\. Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [17]:
#instruction used to load the file
#!wget https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json -P data/ --no-check-certificate
import json
data = json.load(open('data/user_data.json'))
#print(data)

user = pd.read_json("data/user_data.json")
#print(user)

filtered_user = user[user['CreditCardType'] == 'American Express']
#print(filtered_user)

filtered_user.to_csv("filtered_user.csv") 
!type filtered_user.csv

,ID,JobTitle,EmailAddress,FirstNameLastName,CreditCard,CreditCardType
1,2,Investment  Advisor,Clint_Thorpe5003@bulaffy.com,Clint Thorpe,7083-8766-0251-2345,American Express
11,12,Retail Trainee,Phillip_Carpenter9505@famism.biz,Phillip Carpenter,3657-0088-0820-5247,American Express
27,28,Project Manager,Russel_Graves1378@extex.org,Russel Graves,6718-4818-8011-6024,American Express
38,39,Stockbroker,Leanne_Newton1268@typill.biz,Leanne Newton,5438-0816-4166-4847,American Express
56,57,Budget Analyst,Tony_Giles1960@iatim.tech,Tony Giles,8130-3425-7573-7745,American Express
61,62,CNC Operator,Owen_Allcott5125@bauros.biz,Owen Allcott,4156-0107-7210-2630,American Express
67,68,Project Manager,Liam_Lynn3280@kideod.biz,Liam Lynn,7152-3247-6053-2233,American Express
73,74,Dentist,Regina_Woodcock5820@yahoo.com,Regina Woodcock,0208-1753-3870-8002,American Express
80,81,HR Specialist,Carter_Wallace9614@atink.com,Carter Wallace,4256-7201-6717-4322,American Express
91,92,Staffing Consultant,Maia_Star

3\. Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [43]:
url = "https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv?dl=1"
#added ?dl=1 to the url in order to force the download
mush = pd.read_csv(url)
#print(mush)

#print(list(mush.columns))
for i in range(len(list(mush.columns))):
    mushg = mush.groupby(list(mush.columns)[i]).mean()
    #print("Average value of ", list(mush.columns)[i], " is ", mushg)

mush.to_json("mushrooms.json") #used pandas to move the dataframe to json file
#!type mushrooms.json

4\. Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [75]:
#!wget https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat --no-check-certificate
#!type credit_card.dat

with open('credit_card.dat', 'rb') as file:
    file_content = str(file.read())
    file_content = file_content.replace("b'","")
    card_size = 16
    block = 4
    number = 6
    #first_number = file_content[0 : 0+number]
    #print(first_number)
    credit_cards = file_content.split("\\n")
    #print(file_content)
    #print(credit_cards)
    #print(len(credit_cards))
    
    for i in range(len(credit_cards)):
        credit_cards[i] = credit_cards[i][0:114] #removing the last 4 bits from each row
        for j in range(len(0,credit_cards[i]),6):
            
    print(credit_cards)

['110111110110110100111000100000110101110110110111110011100000110011110111110111110101100000110010110010110111110001', '110011110010110101110111100000111000110010110100110111100000110011110011110101110100100000110010110010110110110110', '110010110111110010110010100000110000110000110000110001100000110100110000110001110001100000110110110110110101110010', '110000110110110110110001100000110011110000110110110011100000110011110111110100110010100000110011110001110101110000', '110000110100110011110010100000110001110110110000111000100000110001110100110110110010100000110100110111110100110010', '110101111000110010110111100000110010110000110010110111100000111000110111111000110101100000110111110011110000110011', '110101110111110111110100100000111000110101110010111000100000110010110000111000110111100000110001110001110001110111', '111000110001110100110000100000110001110010110001110000100000110110110011110101110010100000110010111000110100110101', '110101110111110110110100100000110001110001110011110011

5\. **Optional**: load the remote file:

- https://www.dropbox.com/s/aamg1apjhclecka/regression_generated.csv

with Pandas and create a scatter plot with all possible combinations of the following features:
    
  + features_1
  + features_2
  + features_3